# Data Mining Project
### Giorgio Donati, g.donati24@studenti.unipi.it
### Pietro Francaviglia, p.francaviglia1@studenti.unipi.it
#### A.Y. 2021-2022


## Libraries and datasets

In [25]:
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler

from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from pyclustering.cluster.xmeans import xmeans

from col_names import *
from utility import *
from clustering_utility import *

In [26]:
with open(f'{PICKLE_FOLDER}clustering_df.pkl', 'rb') as f:
    clustering_df = pickle.load(f)

with open(f'{PICKLE_FOLDER}pure_clustering_df.pkl', 'rb') as f:
    pure_clustering_df = pickle.load(f)

with open(f'{PICKLE_FOLDER}user_selected_df.pkl', 'rb') as f:
    u_df = pickle.load(f)

with open(f'{PICKLE_FOLDER}pure_user_selected_df.pkl', 'rb') as f:
    pure_u_df = pickle.load(f)

In [27]:
clustering_df_array = StandardScaler().fit_transform(clustering_df)
pure_clustering_df_array = StandardScaler().fit_transform(pure_clustering_df)

In [28]:
# Sum of Squared Errors: cohesion measure
# silhouette score: both cohesion and separation measure
clustering_scores_df = pd.DataFrame(columns = ['silhouette', 'SSE'], dtype=float)

## X-Means

In [29]:
def xmeans_run(array, n_initial_centers, max_centers):
    initial_centers = kmeans_plusplus_initializer(array, n_initial_centers).initialize()
    instance = xmeans(array, initial_centers, max_centers)
    instance.process()
    return instance

In [34]:
xmeans_instance = xmeans_run(clustering_df_array, 2, 10)
xmeans_instance.get_total_wce()

4614.870550994387

In [35]:
pure_xmeans_instance = xmeans_run(pure_clustering_df_array, 2, 10)
pure_xmeans_instance.get_total_wce()

5852.417128373164